# Radar
- [documentation](https://nivo.rocks/radar/)

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='white')
import json

# LOAD FUNCTIONS
%run ../src/useful_functions.ipynb

In [ ]:
# Feelings analysis 
cols = ['Codi_Districte_Educatiu','Nom_Districte_Educatiu', 'f_felicitat', 'f_energia','f_calma', 'f_avorriment','f_estres','f_tristesa']
f_freq = df[cols].copy()
f_freq = f_freq.dropna()

feelings = cols[::-1][:-2]
levels = ['NC']+[str(n) for n in range(0,11)]

def f_dataprep(f_freq, ret_dic=True, codi=0, codi_var=''):
    # Creating a dictionary with the levels to generate stacked bar
    dic = {level:[] for level in levels}

    # Creating a dataframe with the levels for bokeh
    df_ = pd.DataFrame(index=levels, columns=feelings)
    df_ = df_.fillna(np.nan)
   
    for feeling in feelings:
        if codi == 0: 
            ratio = f_freq[feeling].value_counts(normalize=True)*100
            for level in levels: 
                dic[level].append(ratio[level])
                df_[feeling] = ratio

        else:
            ratio = f_freq[f_freq[codi_var] == codi][feeling].value_counts(normalize=True)*100
            for level in ratio.index: 
                dic[level].append(ratio[level])
                df_[feeling] = ratio

    if ret_dic:
        return(dic)
    else:
        return(df_)

In [ ]:
districts = dic_codi_dist(df)
cols = ['f_tristesa', 'f_estres', 'f_avorriment', 'f_calma', 'f_energia', 'f_felicitat']

positive_values = {'f_felicitat': {'value': 'Happiness'},  'f_calma': {'value': 'Calm'}, 'f_energia': {'value': 'Energy'},}
negative_values = {'f_tristesa': {'value': 'Sadness'}, 'f_estres': {'value': 'Stress'}, 'f_avorriment': {'value': 'Boredom'}}

for dist in districts:
    # Generate data for district
    f_df_ = f_dataprep(f_freq, ret_dic=True, codi=dist, codi_var='Codi_Districte_Educatiu')

    # Loop through columns (feelings) these are the items in the lists
    for index, feeling in enumerate(cols):
        sum_values = 0

        # We want only values btw 7 and 10
        for i in range(7,11):
            sum_values += f_df_[str(i)][index]

        dist_name = districts[dist]

        if feeling in positive_values.keys():
            positive_values[feeling][dist_name] = round(sum_values, 2)
        else:
            negative_values[feeling][dist_name] = round(sum_values, 2)

In [ ]:
positive_data, negative_data = [], []
 
for key in positive_values.keys():
   positive_data.append(positive_values[key])

for key in negative_values.keys():
    negative_data.append(negative_values[key])

In [ ]:
with open('../data/radar-positive.json', 'w') as outfile:
    json.dump(positive_data, outfile)

with open('../data/radar-negative.json', 'w') as outfile:
    json.dump(negative_data, outfile)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5cb1a2c0-2f56-40d6-8008-fedeaf8b6a17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>